In [ ]:
# =============================================================================
# COSMIC JOURNEY - A Driving Deep House Track
# =============================================================================
# Self-contained song with:
# - Groovy Rhodes chords with SWING for human feel (not robotic!)
# - SYNTH arpeggios as THE DROP (star of the chorus!)
# - Choir and synth NEVER play together (clean separation)
# - Professional build-ups with kick dropout, snare rolls, risers
# - Synth resolves naturally down to root (not repeated notes)
# - Full verse/chorus/breakdown structure
# - MIDI export for conversion to WAV
# =============================================================================

from scamp import *
import random
import os
import subprocess
from pathlib import Path
from types import SimpleNamespace

# --- WAV EXPORT HELPER ---
def find_scamp_soundfont():
    """Find the soundfont that SCAMP uses for playback."""
    import scamp
    
    # First check SCAMP's bundled soundfont (this is what it actually uses!)
    scamp_dir = Path(scamp.__file__).parent / "soundfonts"
    if scamp_dir.exists():
        for sf in scamp_dir.glob("*.sf2"):
            return str(sf)
    
    # Check user SCAMP directory
    user_scamp = Path.home() / ".scamp" / "soundfonts"
    if user_scamp.exists():
        for sf in user_scamp.glob("*.sf2"):
            return str(sf)
    
    # Check common system locations
    common_paths = [
        "/opt/homebrew/Cellar/fluid-synth/2.5.1/share/fluid-synth/sf2/VintageDreamsWaves-v2.sf2",
        "/usr/share/sounds/sf2/FluidR3_GM.sf2",
        "/usr/share/soundfonts/FluidR3_GM.sf2",
    ]
    for p in common_paths:
        if os.path.exists(p):
            return p
    return None

def fix_midi_programs(midi_file):
    """Add program change messages to MIDI file so FluidSynth uses correct instruments."""
    try:
        import mido
    except ImportError:
        print("   ⚠️ mido not installed! Run in terminal: pip3 install mido")
        print("   ⚠️ Then RESTART the Jupyter kernel (Kernel → Restart)")
        return False
    
    # SCAMP creates tracks in order we create instruments:
    # Track 0: tempo/meta, Track 1: drums, Track 2: bass, Track 3: thump, 
    # Track 4: synth, Track 5: lead, Track 6: choir
    # HARDCODED presets to avoid scope issues with constants
    # Map track INDEX to (program, channel)
    track_config = {
        1: (0, 9),     # drums: Standard Kit on channel 10 (9 in 0-index)
        2: (38, 0),    # bass: Synth Bass 1
        3: (18, 1),    # thump: Organ 3
        4: (81, 2),    # synth: Saw Wave
        5: (5, 3),     # lead: E.Piano 2
        6: (52, 4),    # choir: Choir Aahs
    }
    
    mid = mido.MidiFile(midi_file)
    new_tracks = []
    
    for i, track in enumerate(mid.tracks):
        if i in track_config:
            program, channel = track_config[i]
            
            new_track = mido.MidiTrack()
            # Add program change at time 0
            new_track.append(mido.Message('program_change', channel=channel, program=program, time=0))
            
            # Copy all messages, updating channel for note events
            for msg in track:
                if hasattr(msg, 'channel'):
                    msg = msg.copy(channel=channel)
                new_track.append(msg)
            new_tracks.append(new_track)
        else:
            new_tracks.append(track)
    
    mid.tracks = new_tracks
    mid.save(midi_file)
    print(f"   🔧 Fixed MIDI: assigned programs to {len(track_config)} instrument tracks")
    return True

def midi_to_wav(midi_file, wav_file):
    """Convert MIDI to WAV using the same soundfont SCAMP uses."""
    soundfont = find_scamp_soundfont()
    
    if not soundfont:
        print("   ⚠️  No soundfont found. Install FluidSynth soundfont:")
        print("      brew install fluid-synth")
        return False
    
    print(f"   🎵 Using soundfont: {soundfont}")
    
    # FluidSynth 2.5+ syntax: options before soundfont and midi file
    try:
        result = subprocess.run([
            "fluidsynth", "-ni",
            "-F", wav_file,      # Output file BEFORE soundfont
            "-T", "wav",         # Explicitly set WAV format
            soundfont,           # Soundfont
            midi_file            # MIDI file last
        ], capture_output=True, text=True, timeout=300)
        
        if result.returncode == 0 and os.path.exists(wav_file):
            return True
        else:
            print(f"   ⚠️  FluidSynth error: {result.stderr}")
    except FileNotFoundError:
        print("   ⚠️  FluidSynth not found. Install with:")
        print("      brew install fluid-synth")
    except Exception as e:
        print(f"   ⚠️  Error: {e}")
    
    return False

# --- PITCH UTILITIES ---
NOTES_PER_OCTAVE = 12
CHROMATIC_SCALE = ['C', 'Cs', 'D', 'Ds', 'E', 'F', 'Fs', 'G', 'Gs', 'A', 'As', 'B']
FLAT_TO_SHARP = {'Db': 'Cs', 'Eb': 'Ds', 'Gb': 'Fs', 'Ab': 'Gs', 'Bb': 'As'}

def build_pitches():
    """Build namespace of all MIDI pitches (N.C4, N.Fs4, N.Bb3, etc.)"""
    pitches = {}
    for octave in range(-1, 10):
        for pos, name in enumerate(CHROMATIC_SCALE):
            midi = (octave + 1) * 12 + pos
            if 0 <= midi <= 127:
                label = 'm1' if octave == -1 else str(octave)
                pitches[f'{name}{label}'] = midi
        for flat, sharp in FLAT_TO_SHARP.items():
            label = 'm1' if octave == -1 else str(octave)
            if f'{sharp}{label}' in pitches:
                pitches[f'{flat}{label}'] = pitches[f'{sharp}{label}']
    return SimpleNamespace(**pitches)

N = build_pitches()

# --- SONG CONSTANTS ---
SONG_TEMPO = 122

# Presets
DRUM_BANK = 128
DRUM_PRESET = 0
BASS_PRESET = 38       # Synth Bass 1 - gritty aggressive
THUMP_PRESET = 18      # Organ 3
SYNTH_PRESET = 81      # Saw Wave
LEAD_PRESET = 5        # E.Piano 2 - warm, groovy Rhodes/Wurlitzer sound
CHOIR_PRESET = 52      # Choir Aahs

# Timing
SIXTEENTH = 0.25
EIGHTH = 0.5
QUARTER = 1.0
BAR = 4.0

# --- MASTERED VOLUME LEVELS ---
# Lead drives verses, Synth drops in chorus, Choir for breakdown only
MIX = {
    'kick': 0.9,      # Foundation - not overpowering
    'snare': 0.6,     # Punchy but balanced
    'hihat': 0.4,     # Supportive
    'bass': 0.6,      # Warm foundation
    'thump': 0.45,    # Space organ warmth
    'synth': 0.55,    # Balanced - not overwhelming
    'lead': 0.6,      # Rhodes chords - smooth
    'choir': 0.4,     # Ethereal (breakdown/outro only)
    'riser': 0.4,     # Subtle build-up tension
}

CRASH = 49  # Crash cymbal

# --- DRUM PATTERNS ---
KICK_PATTERN = [1, 0, 1, 0, 1, 0, 1, 0]
SNARE_PATTERN = [0, 0, 1, 0, 0, 0, 1, 0]
HIHAT_OPEN = [0, 0, 0, 1, 0, 0, 0, 1]

# --- BASS PATTERN (2 bars) ---
# Aggressive root-heavy groove
BASS_NOTES = [
    N.C2, None, N.C2, N.C2, None, N.C2, N.G1, None,
    N.C2, N.C2, None, N.Eb2, N.C2, None, N.C2, N.C2,
]

# --- THUMP PATTERN (2 bars) ---
THUMP_NOTES = [
    N.C2, None, N.Eb2, N.G2, None, N.C3, N.Bb2, N.G2,
    N.C2, N.Eb2, N.G2, None, N.C3, N.Bb2, N.G2, N.C2,
]

# --- SYNTH ARPEGGIO (2 bars) ---
# Earwormy hook: mostly 16ths with strategic rests for groove
# Ascending energy → breathing → descent → resolve
SYNTH_NOTES = [
    N.C4, N.G4, N.C5, N.Eb5, N.G5, N.Eb5, N.C5, N.G4,    # Bar 1a: rising 16ths
    N.C5, N.Eb5, N.G5, None, N.C6, N.G5, N.Eb5, None,    # Bar 1b: peak + breath
    N.C5, N.G4, N.Eb5, N.C5, N.Bb4, N.G4, None, N.Eb4,   # Bar 2a: descending
    N.C4, N.G4, N.C5, None, N.G4, N.Eb4, N.C4, None,     # Bar 2b: resolve home
]

# --- LEAD CHORDS ---
# Groovy Rhodes chords - warm, soulful, with SWING for human feel!
# 2-bar chord pattern that grooves - stabs on offbeats for house feel
Cm7_lead = [N.C4, N.Eb4, N.G4, N.Bb4]
Fm7_lead = [N.F3, N.Ab3, N.C4, N.Eb4]
AbMaj7_lead = [N.Ab3, N.C4, N.Eb4, N.G4]
Gm7_lead = [N.G3, N.Bb3, N.D4, N.F4]

# Pattern: chord on beat, then offbeat stabs (classic house piano)
# Format: (chord_or_None, swing_delay) - swing_delay adds humanization
LEAD_CHORDS = [
    # Bar 1: Cm7 groove
    (Cm7_lead, 0), (None, 0), (Cm7_lead, 0.02), (None, 0),
    (None, 0), (Cm7_lead, 0.03), (None, 0), (Cm7_lead, 0.02),
    # Bar 2: AbMaj7 answer
    (AbMaj7_lead, 0), (None, 0), (None, 0), (AbMaj7_lead, 0.03),
    (None, 0), (AbMaj7_lead, 0.02), (Gm7_lead, 0), (None, 0),
]

# --- CHOIR CHORDS (1 per bar) ---
CHOIR_CHORDS = [
    [N.C4, N.Eb4, N.G4],      # Cm
    [N.Ab3, N.C4, N.Eb4],     # Ab
    [N.Bb3, N.D4, N.F4],      # Bb
    [N.G3, N.Bb3, N.D4],      # Gm
]

# --- SONG STRUCTURE ---
# SYNTH is THE DROP! Choir and synth NEVER play together.
# Lead = groovy Rhodes chords in verses
# Synth = the star of choruses  
# Choir = ethereal pad in breakdown/outro only
SONG_STRUCTURE = [
    # INTRO - GRADUAL build
    ('intro_1', 4, {'kick': 0.7, 'hihat': 0.4}),
    ('intro_2', 4, {'kick': 0.8, 'hihat': 0.5, 'snare': 0.4, 'bass': 0.6, 'lead': 0.6}),
    # VERSE 1: Lead at SAME volume as intro_2 (0.6) - no jarring change
    ('verse', 4, {'kick': 0.85, 'snare': 0.6, 'hihat': 0.5, 'bass': 0.7, 'thump': 0.4, 'lead': 0.6}),
    ('verse_choir', 4, {'kick': 0.85, 'snare': 0.6, 'hihat': 0.5, 'bass': 0.7, 'thump': 0.4, 'lead': 0.6, 'choir': 0.3}),
    ('buildup', 4, {'kick': 0.8, 'hihat': 0.6, 'bass': 0.7, 'thump': 0.3, 'snare_roll': 1}),
    # CHORUS 1: SYNTH DROP
    ('chorus', 8, {'kick': 1, 'snare': 0.7, 'hihat': 0.6, 'bass': 0.8, 'thump': 0.5, 'synth': 0.7}),
    # VERSE 2: Same lead volume
    ('verse', 4, {'kick': 0.85, 'snare': 0.6, 'hihat': 0.5, 'bass': 0.7, 'thump': 0.5, 'lead': 0.6}),
    ('verse_choir', 4, {'kick': 0.85, 'snare': 0.6, 'hihat': 0.5, 'bass': 0.7, 'thump': 0.5, 'lead': 0.6, 'choir': 0.35}),
    ('buildup', 4, {'kick': 0.8, 'hihat': 0.6, 'bass': 0.7, 'thump': 0.4, 'snare_roll': 1}),
    # CHORUS 2: Bigger synth
    ('chorus', 8, {'kick': 1, 'snare': 0.7, 'hihat': 0.6, 'bass': 0.8, 'thump': 0.6, 'synth': 0.75}),
    # BREAKDOWN: Gentle - choir with light drums
    ('breakdown', 4, {'kick': 0.3, 'hihat': 0.2, 'choir': 0.5}),
    ('buildup', 4, {'kick': 0.7, 'hihat': 0.5, 'bass': 0.5, 'snare_roll': 1}),
    # FINAL CHORUS: Full synth
    ('chorus', 8, {'kick': 1, 'snare': 0.7, 'hihat': 0.6, 'bass': 0.9, 'thump': 0.6, 'synth': 0.8}),
    # OUTRO: Gentle fade with choir
    ('outro', 4, {'kick': 0.5, 'hihat': 0.3, 'bass': 0.4, 'thump': 0.2, 'choir': 0.25}),
]

# --- SESSION AND INSTRUMENTS (globals set in play_cosmic_journey) ---
song = None
drums = None
bass_inst = None
thump_inst = None
synth_inst = None
lead_inst = None
choir_inst = None

# --- PLAYBACK FUNCTIONS ---
def play_section(section_name, bars, instruments):
    print(f"  [{section_name.upper()}] - {bars} bars")
    for bar in range(bars):
        play_bar(bar, instruments, section_name, total_bars=bars)

def play_bar(bar_num, instruments, section, total_bars=4):
    for sixteenth in range(16):
        play_sixteenth_note(bar_num, sixteenth, instruments, section, total_bars)

def play_sixteenth_note(bar_num, sixteenth, instruments, section, total_bars=4):
    eighth = sixteenth // 2
    is_eighth = (sixteenth % 2 == 0)
    is_offbeat_16th = (sixteenth % 2 == 1)
    global_eighth = bar_num * 8 + eighth
    global_sixteenth = bar_num * 16 + sixteenth
    buildup_progress = (bar_num * 16 + sixteenth) / (total_bars * 16)
    
    # --- DRUMS (skip during build-up) ---
    if 'kick' in instruments and KICK_PATTERN[eighth] and is_eighth and 'snare_roll' not in instruments:
        drums.play_note(36, MIX['kick'] * instruments['kick'], EIGHTH * 0.9, blocking=False)
    
    if 'snare' in instruments and SNARE_PATTERN[eighth] and is_eighth and 'snare_roll' not in instruments:
        drums.play_note(38, MIX['snare'] * instruments['snare'], EIGHTH, blocking=False)
    
    # --- BUILD-UP ---
    if 'snare_roll' in instruments:
        # Kick dropout
        if 'kick' in instruments and KICK_PATTERN[eighth] and is_eighth:
            if bar_num == 0:
                drums.play_note(36, MIX['kick'] * 0.8, EIGHTH * 0.9, blocking=False)
            elif bar_num == 1:
                drums.play_note(36, MIX['kick'] * 0.5, EIGHTH * 0.9, blocking=False)
        
        # Snare roll
        snare_vol = 0.4 + (buildup_progress * 0.6)
        play_roll = False
        if bar_num == 0:
            play_roll = SNARE_PATTERN[eighth] and is_eighth
        elif bar_num == 1:
            play_roll = is_eighth and (eighth % 2 == 0)
        elif bar_num == 2:
            play_roll = is_eighth
        else:
            play_roll = sixteenth < 12
        
        if play_roll:
            drums.play_note(38, MIX['snare'] * snare_vol, SIXTEENTH * 0.7, blocking=False)
        
        # Crash on final 16th
        if bar_num == total_bars - 1 and sixteenth == 15:
            drums.play_note(CRASH, 1.0, QUARTER, blocking=False)
        
        # Hi-hat intensification
        hihat_vol = MIX['hihat'] * (0.5 + buildup_progress * 0.5)
        if bar_num < 2:
            if is_eighth:
                drums.play_note(42, hihat_vol, EIGHTH, blocking=False)
        else:
            if sixteenth < 12 or bar_num < total_bars - 1:
                drums.play_note(42, hihat_vol, SIXTEENTH, blocking=False)
        
        # Riser synth
        if sixteenth == 0:
            riser_vol = MIX['riser'] * buildup_progress * 0.8
            riser_note = N.G5 if bar_num < 2 else N.C6
            synth_inst.play_note(riser_note, riser_vol, BAR * 0.95, blocking=False)
    
    # --- HI-HATS (normal) ---
    if 'hihat' in instruments and 'snare_roll' not in instruments:
        vol = MIX['hihat'] * instruments['hihat']
        if HIHAT_OPEN[eighth] and is_eighth:
            drums.play_note(46, vol * 0.7, EIGHTH, blocking=False)
        elif is_offbeat_16th:
            drums.play_note(42, vol * 0.5, SIXTEENTH, blocking=False)
        else:
            drums.play_note(42, vol, SIXTEENTH, blocking=False)
    
    # --- BASS ---
    if 'bass' in instruments and is_eighth:
        note = BASS_NOTES[global_eighth % len(BASS_NOTES)]
        if note:
            vol = MIX['bass'] * instruments['bass'] * (1.05 if eighth % 2 == 1 else 0.95)
            bass_inst.play_note(note, vol, EIGHTH * 0.85, blocking=False)
    
    # --- THUMP ---
    if 'thump' in instruments and is_eighth:
        note = THUMP_NOTES[global_eighth % len(THUMP_NOTES)]
        if note:
            thump_inst.play_note(note, MIX['thump'] * instruments['thump'], EIGHTH * 1.1, blocking=False)
    
    # --- SYNTH (THE DROP!) ---
    if 'synth' in instruments:
        note = SYNTH_NOTES[global_sixteenth % len(SYNTH_NOTES)]
        if note:
            vol = MIX['synth'] * instruments['synth'] * random.uniform(0.85, 1.0)
            synth_inst.play_note(note, vol, SIXTEENTH * 0.9, blocking=False)
    
    # --- LEAD CHORDS (groovy Rhodes with swing!) ---
    if 'lead' in instruments and is_eighth:
        chord_data = LEAD_CHORDS[global_eighth % len(LEAD_CHORDS)]
        chord, swing = chord_data
        if chord:
            # Apply swing delay for human feel
            if swing > 0:
                song.wait(swing)
            # Velocity variation for groove
            vel_variation = random.uniform(0.85, 1.0)
            # Offbeats slightly softer for groove
            is_offbeat = (global_eighth % 2 == 1)
            vol = MIX['lead'] * instruments['lead'] * vel_variation * (0.9 if is_offbeat else 1.0)
            # Play chord
            for note in chord:
                lead_inst.play_note(note, vol, EIGHTH * 0.7, blocking=False)
    
    # --- CHOIR ---
    if 'choir' in instruments and sixteenth == 0:
        chord = CHOIR_CHORDS[bar_num % len(CHOIR_CHORDS)]
        vol = MIX['choir'] * instruments['choir']
        for note in chord:
            choir_inst.play_note(note, vol, BAR * 0.95, blocking=False)
    
    song.wait(SIXTEENTH)

# --- PLAY THE SONG ---
def play_cosmic_journey(record=False):
    global song, drums, bass_inst, thump_inst, synth_inst, lead_inst, choir_inst
    
    print("=" * 60)
    print("🎹✨ COSMIC JOURNEY ✨🎹")
    print("   A Driving Deep House Track")
    print("=" * 60)
    print(f"   Tempo: {SONG_TEMPO} BPM | Key: C minor")
    print(f"   Lead: Groovy Rhodes chords with swing!")
    print(f"   DROP: SYNTH arpeggios - the star of the chorus!")
    print(f"   Choir: Ethereal pad (breakdown/outro only - never with synth)")
    
    total_bars = sum(s[1] for s in SONG_STRUCTURE)
    duration_sec = total_bars * 4 * 60 / SONG_TEMPO
    print(f"   Duration: {total_bars} bars (~{int(duration_sec // 60)}:{int(duration_sec % 60):02d})")
    print("=" * 60)
    
    # Create session and instruments
    song = Session(tempo=SONG_TEMPO)
    drums = song.new_part("drums", preset=(DRUM_BANK, DRUM_PRESET))
    bass_inst = song.new_part("bass", preset=(0, BASS_PRESET))
    thump_inst = song.new_part("thump", preset=(0, THUMP_PRESET))
    synth_inst = song.new_part("synth", preset=(0, SYNTH_PRESET))
    lead_inst = song.new_part("lead", preset=(0, LEAD_PRESET))
    choir_inst = song.new_part("choir", preset=(0, CHOIR_PRESET))
    
    if record:
        print("   🎙️ Recording to MIDI...")
        song.start_transcribing()
        # Prime each instrument with a silent note to ensure program changes are in MIDI
        for inst in [drums, bass_inst, thump_inst, synth_inst, lead_inst, choir_inst]:
            inst.play_note(60, 0.01, 0.01, blocking=False)  # Inaudible primer
        song.wait(0.05)  # Brief pause after primers
    
    for section_name, bars, instruments in SONG_STRUCTURE:
        play_section(section_name, bars, instruments)
    
    print("=" * 60)
    print("🌟 Journey complete!")
    print("=" * 60)
    
    if record:
        performance = song.stop_transcribing()
        midi_file = "cosmic_journey.mid"
        wav_file = "cosmic_journey.wav"
        
        performance.export_to_midi_file(midi_file)
        print(f"   💾 MIDI saved: {midi_file}")
        
        # Fix MIDI program changes so FluidSynth uses correct instruments
        fix_midi_programs(midi_file)
        
        # Convert to WAV using same soundfont as playback
        print("   🔄 Converting to WAV...")
        if midi_to_wav(midi_file, wav_file):
            file_size = os.path.getsize(wav_file) / (1024 * 1024)
            print(f"   ✅ WAV saved: {wav_file} ({file_size:.1f} MB)")
            print("   🎧 Open with: open cosmic_journey.wav")
            return wav_file
        else:
            print(f"   📝 Manual conversion: fluidsynth -ni soundfont.sf2 {midi_file} -F {wav_file}")
            return midi_file
    return None

# --- RUN ---
play_cosmic_journey(record=True)


🎹✨ COSMIC JOURNEY ✨🎹
   A Driving Deep House Track
   Tempo: 122 BPM | Key: C minor
   Lead: Groovy Rhodes chords with swing!
   DROP: SYNTH arpeggios - the star of the chorus!
   Choir: Ethereal pad (breakdown/outro only - never with synth)
   Duration: 68 bars (~2:13)
   🎙️ Recording to MIDI...
  [INTRO_1] - 4 bars
  [INTRO_2] - 4 bars
  [VERSE] - 4 bars
  [VERSE_CHOIR] - 4 bars


  [BUILDUP] - 4 bars
  [CHORUS] - 8 bars
  [VERSE] - 4 bars
  [VERSE_CHOIR] - 4 bars
  [BUILDUP] - 4 bars
  [CHORUS] - 8 bars
  [BREAKDOWN] - 4 bars
  [BUILDUP] - 4 bars
  [CHORUS] - 8 bars


  [OUTRO] - 4 bars
🌟 Journey complete!
   💾 MIDI saved: cosmic_journey.mid
   🔧 Fixed MIDI: assigned programs to 6 instrument tracks
   🔄 Converting to WAV...
   🎵 Using soundfont: /Users/bsgreenb/Library/Python/3.11/lib/python/site-packages/scamp/soundfonts/Merlin.sf2
   ✅ WAV saved: cosmic_journey.wav (23.8 MB)
   🎧 Open with: open cosmic_journey.wav


'cosmic_journey.wav'

In [4]:
from scamp import Session

# 122 BPM is a classic Deep House tempo
s = Session(tempo=122)

# Use the custom Lately_Bass soundfont
bass = s.new_part("Lately Bass", soundfont="Lately_Bass.sf2")

print("🎹 Playing Deep House Groove...")

# MIDI Notes: C2=36, Eb2=39, F2=41, G2=43, Bb2=46
# A punchy, syncopated groove
def play_groove():
    # Velocity variation is key for groove
    # Bar 1
    bass.play_note(36, 0.9, 0.75)   # C2 (dotted 8th)
    bass.play_note(36, 0.6, 0.25)   # C2 ghost (16th)
    s.wait(0.25)                    # Rest (16th)
    bass.play_note(39, 0.8, 0.25)   # Eb2 (16th)
    bass.play_note(41, 0.85, 0.5)   # F2 (8th)
    s.wait(0.5)                     # Rest (8th) - creates space
    bass.play_note(43, 0.8, 0.5)    # G2 (8th)
    bass.play_note(39, 0.7, 0.5)    # Eb2 (8th) slide down
    
    # Bar 2 (variation)
    bass.play_note(36, 0.9, 0.75)   # C2
    bass.play_note(36, 0.6, 0.25)   # C2 ghost
    s.wait(0.5)
    bass.play_note(46, 0.75, 0.25)  # Bb2 (16th)
    bass.play_note(43, 0.8, 0.25)   # G2 (16th)
    bass.play_note(41, 0.7, 0.5)    # F2 (8th)
    bass.play_note(39, 0.8, 0.5)    # Eb2 (8th)
    bass.play_note(36, 0.6, 1.0)    # C2 (quarter)

# Loop it twice
for _ in range(2):
    play_groove()


Using preset Lately Bass V1 for Lately Bass
🎹 Playing Deep House Groove...
